In [1]:
import boto3
import pandas as pd
import numpy as np
import re
from io import StringIO
import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [2]:
# Nazwa bucketu S3
BUCKET_NAME = "zadmod-9"

# Wczytanie zmiennych środowiskowych
load_dotenv()

# Inicjalizacja klienta S3
s3 = boto3.client("s3")

def load_data_from_s3(file_key):
    """Wczytuje dane z S3 do DataFrame."""
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=file_key)
    csv_data = obj['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_data), sep=";")
    return df

# Wczytywanie danych dla 2023 i 2024
wroclaw_2023_df = load_data_from_s3("Dane_mod9/halfmarathon_wroclaw_2023__final.csv")
wroclaw_2024_df = load_data_from_s3("Dane_mod9/halfmarathon_wroclaw_2024__final.csv")


In [3]:
# I. KONWERSJA CZASU
def convert_time_to_seconds(time):
    if pd.isnull(time) or time in ['DNS', 'DNF']: #DID NOT START / DID NOT FINISH
        return None
    time = time.split(':')
    return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])


In [4]:
# Usuwanie zbędnych kolumn
def get_cleaned_dataframe(df_2023: pd.DataFrame, df_2024: pd.DataFrame) -> pd.DataFrame:
    df_combined = pd.concat([df_2023, df_2024], ignore_index=True)  # Łączenie danych z obu lat
    df_model = df_combined[df_combined['Miejsce'].notnull()].copy() # Usuwanie wierszy z brakującymi miejscami 

    # Usuwanie zbędnych kolumn
    columns_to_drop = ['Drużyna', 'Miasto'] # Wybrane kolumny do usunięcia
    df_model.drop(columns=columns_to_drop, inplace=True)

    return df_model


In [5]:
cleaned_df = get_cleaned_dataframe(wroclaw_2023_df, wroclaw_2024_df)
missing_values_count = cleaned_df.isna().sum().reset_index(name='ilość')


In [6]:
cleaned_df.isna().sum().reset_index(name='ilość')


,index,ilość
0,Miejsce,0
1,Numer startowy,0
2,Imię,0
3,Nazwisko,0
4,Kraj,0
5,Płeć,0
6,Płeć Miejsce,0
7,Kategoria wiekowa,20
8,Kategoria wiekowa Miejsce,20
9,Rocznik,485


In [7]:
# Uzupełnia brakujące wartości w kolumnie 'Rocznik' medianą dla każdej 'Kategorii wiekowej' i dodaje kolumnę 'Wiek
def impute_rocznik_from_category(df: pd.DataFrame) -> pd.DataFrame:
    """
    Uzupełnia brakujące wartości w kolumnie 'Rocznik'
    medianą dla każdej 'Kategorii wiekowej' i dodaje kolumnę 'Wiek'.
    """
    if 'Rocznik' not in df.columns or 'Kategoria wiekowa' not in df.columns:
        return df

    df_result = df.copy()

    # Oblicz medianę 'Rocznik' dla każdej kategorii wiekowej
    median_map = df_result.groupby('Kategoria wiekowa')['Rocznik'].transform('median').round(0).astype('Int64')

    # Uzupełnij brakujące 'Rocznik' medianą odpowiedniej kategorii
    df_result['Rocznik'] = df_result['Rocznik'].fillna(median_map)

    # Dodaj kolumnę 'Wiek'
    df_result['Wiek'] = 2024 - df_result['Rocznik']

    # Usuń kolumnę 'Rocznik'
    df_result = df_result.drop(columns=['Rocznik'])

    # Wyświetl liczby brakujących przed i po
    print(f"Brakujące wartości w 'Rocznik' przed imputacją: {(df['Rocznik'].isnull().sum())}")
    print(f"Brakujące wartości w 'Rocznik' po imputacji: {(df_result['Wiek'].isnull().sum())}")

    return df_result


In [8]:
cleaned_df = get_cleaned_dataframe(wroclaw_2023_df, wroclaw_2024_df)
cleaned_df = impute_rocznik_from_category(cleaned_df)
cleaned_df


Brakujące wartości w 'Rocznik' przed imputacją: 485
Brakujące wartości w 'Rocznik' po imputacji: 0


,Miejsce,Numer startowy,Imię,Nazwisko,Kraj,Płeć,Płeć Miejsce,Kategoria wiekowa,Kategoria wiekowa Miejsce,5 km Czas,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Wiek
0,1.0,1787,TOMASZ,GRYCKO,POL,M,1.0,M30,1.0,00:14:37,...,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509,32.0
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,POL,M,2.0,M30,2.0,00:14:48,...,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875,38.0
2,3.0,3832,KRZYSZTOF,HADAS,POL,M,3.0,M20,1.0,00:15:46,...,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475,28.0
3,4.0,416,DAMIAN,DYDUCH,POL,M,4.0,M30,3.0,00:16:11,...,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963,36.0
4,5.0,8476,KAMIL,MAŃKOWSKI,POL,M,5.0,M20,2.0,00:16:12,...,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19245,10298.0,898,EWA KATARZYNA,KASIERSKA,POL,K,3059.0,K70,6.0,00:43:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:19:47,9.470649,76.0
19246,10299.0,10456,MAGDALENA,PRYNDA,POL,K,3060.0,K40,1030.0,00:42:06,...,02:15:28,10301.0,9.580000,03:10:19,10303.0,10.970000,0.162733,03:21:08,9.534645,46.0
19247,10300.0,11311,HELENA,KOLANY,POL,K,3061.0,K60,48.0,00:49:20,...,02:23:22,10303.0,9.010000,03:11:34,10304.0,9.640000,-0.029333,03:22:18,9.589950,69.0
19248,10301.0,8404,JOANNA,KAJDANEK,POL,K,3062.0,K40,1031.0,00:46:48,...,02:26:03,10304.0,9.980000,03:17:25,10305.0,10.273333,0.057000,03:28:32,9.885439,43.0


In [9]:
# Zmiana kolumn czasu w cleaned_df - Konwersja czasu na sekundy
time_columns = ['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas']
cleaned_df[time_columns] = cleaned_df[time_columns].applymap(convert_time_to_seconds)

# Teraz wyświetlasz
print(cleaned_df)


       Miejsce  Numer startowy           Imię       Nazwisko Kraj Płeć  \
0          1.0            1787         TOMASZ         GRYCKO  POL    M   
1          2.0               3      ARKADIUSZ  GARDZIELEWSKI  POL    M   
2          3.0            3832      KRZYSZTOF          HADAS  POL    M   
3          4.0             416         DAMIAN         DYDUCH  POL    M   
4          5.0            8476          KAMIL      MAŃKOWSKI  POL    M   
...        ...             ...            ...            ...  ...  ...   
19245  10298.0             898  EWA KATARZYNA      KASIERSKA  POL    K   
19246  10299.0           10456      MAGDALENA         PRYNDA  POL    K   
19247  10300.0           11311         HELENA         KOLANY  POL    K   
19248  10301.0            8404         JOANNA       KAJDANEK  POL    K   
19249  10302.0           11155      Anonimowy       ZAWODNIK  POL    M   

       Płeć Miejsce Kategoria wiekowa  Kategoria wiekowa Miejsce  5 km Czas  \
0               1.0             

C:\Users\Michal\AppData\Local\Temp\ipykernel_18852\3363593751.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df[time_columns] = cleaned_df[time_columns].applymap(convert_time_to_seconds)


In [10]:
# Usuwanie brakujących wartości
cleaned_df.dropna(subset=time_columns, inplace=True)
print(f"Liczba rekordów po usunięciu brakujących: {len(cleaned_df)}")


Liczba rekordów po usunięciu brakujących: 18377


In [11]:
cleaned_df.isna().sum().reset_index(name='ilość')


,index,ilość
0,Miejsce,0
1,Numer startowy,0
2,Imię,0
3,Nazwisko,0
4,Kraj,0
5,Płeć,0
6,Płeć Miejsce,0
7,Kategoria wiekowa,17
8,Kategoria wiekowa Miejsce,17
9,5 km Czas,0


In [12]:
# Usuwanie outliers

# Funkcja do policzenia liczby outliers w serii
def count_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return len(outliers)

# Funkcja do usuwania outliers z DataFrame w podanej kolumnie
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)].copy()

# Lista kolumn, z których chcesz usunąć outliers
columns_to_clean = [
    'Tempo', 
    '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas',
    '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo'
]

# Liczba outliers przed usunięciem
outliers_counts = {}
for col in columns_to_clean:
    if col in cleaned_df.columns and pd.api.types.is_numeric_dtype(cleaned_df[col]):
        outliers_counts[col] = count_outliers(cleaned_df[col])

print("--- Liczba outliers przed usunięciem ---")
for col, count in outliers_counts.items():
    print(f"{col}: {count}")

# Usuwanie outliers kaskadowo
for col in columns_to_clean:
    if col in cleaned_df.columns and pd.api.types.is_numeric_dtype(cleaned_df[col]):
        cleaned_df = remove_outliers(cleaned_df, col)

print(f"\nLiczba rekordów po usunięciu outliers: {len(cleaned_df)}")


--- Liczba outliers przed usunięciem ---
Tempo: 222
5 km Czas: 135
10 km Czas: 178
15 km Czas: 203
20 km Czas: 225
Czas: 222
5 km Tempo: 135
10 km Tempo: 256
15 km Tempo: 294
20 km Tempo: 408

Liczba rekordów po usunięciu outliers: 17626


In [13]:
# ========================================
# PRZYGOTOWANIE UNIWERSALNEGO model_data
# ========================================

print("\n" + "="*60)
print("PRZYGOTOWANIE UNIWERSALNYCH DANYCH DO MODELU")
print("="*60)

# Stwórz model_data z WSZYSTKIMI potencjalnie przydatnymi kolumnami
model_data = cleaned_df[[
    # Cechy podstawowe
    '5 km Czas', 
    '10 km Czas', 
    '15 km Czas',
    '20 km Czas',
    'Wiek', 
    'Płeć',
    
    # Target
    'Czas'
]].copy()


print(f"📊 Uniwersalne dane: {len(model_data)} rekordów")
print(f"📋 Dostępne kolumny: {list(model_data.columns)}")
print(f"\nPierwsze wiersze:")
print(model_data.head())
print(f"\nStatystyki:")
print(model_data.describe())

# Sprawdź korelacje - TYLKO dla kolumn numerycznych
print("\n📈 Korelacje z czasem końcowym:")
numeric_columns = model_data.select_dtypes(include=[np.number]).columns  # Tylko numeryczne
correlations = model_data[numeric_columns].corr()['Czas'].sort_values(ascending=False)
print(correlations)

# Sprawdź rozkład płci
print("\n👥 Rozkład płci:")
print(model_data['Płeć'].value_counts())
print(f"\nŚredni czas wg płci:")
print(model_data.groupby('Płeć')['Czas'].agg(['mean', 'std', 'count']))

# ========================================
# ANALIZA WIEKU W GRUPACH
# ========================================

print("\n📊 Analiza wieku:")
age_groups = pd.cut(model_data['Wiek'], 
                    bins=[0, 30, 40, 50, 60, 100],
                    labels=['<30', '30-40', '40-50', '50-60', '60+'])
age_analysis = model_data.groupby(age_groups)['Czas'].agg(['mean', 'std', 'count'])
print(age_analysis)



PRZYGOTOWANIE UNIWERSALNYCH DANYCH DO MODELU
📊 Uniwersalne dane: 17626 rekordów
📋 Dostępne kolumny: ['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Wiek', 'Płeć', 'Czas']

Pierwsze wiersze:
    5 km Czas  10 km Czas  15 km Czas  20 km Czas  Wiek Płeć  Czas
8      1030.0      2063.0      3131.0      4263.0  23.0    M  4456
10     1029.0      2063.0      3131.0      4282.0  27.0    M  4483
11     1035.0      2063.0      3133.0      4284.0  32.0    M  4490
12     1035.0      2063.0      3133.0      4290.0  28.0    M  4496
13     1036.0      2064.0      3134.0      4292.0  33.0    M  4505

Statystyki:
          5 km Czas    10 km Czas    15 km Czas    20 km Czas       Wiek  \
count  17626.000000  17626.000000  17626.000000  17626.000000    17626.0   
mean    1658.139113   3306.438443   5048.929536   6915.312833  41.071145   
std      227.180240    462.458806    720.477079   1014.010811  56.812103   
min     1017.000000   2014.000000   3060.000000   4135.000000       18.0   
25%  

C:\Users\Michal\AppData\Local\Temp\ipykernel_18852\3088311681.py:51: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_analysis = model_data.groupby(age_groups)['Czas'].agg(['mean', 'std', 'count'])


In [14]:
cleaned_df


,Miejsce,Numer startowy,Imię,Nazwisko,Kraj,Płeć,Płeć Miejsce,Kategoria wiekowa,Kategoria wiekowa Miejsce,5 km Czas,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Wiek
8,9.0,5657,CYPRIAN,GRZELKA,POL,M,9.0,M20,4.0,1030.0,...,3131.0,12.0,3.560000,4263.0,10.0,3.773333,0.022733,4456,3.520581,23.0
10,11.0,7270,MATEUSZ,POŁCZYŃSKI,POL,M,11.0,M20,5.0,1029.0,...,3131.0,13.0,3.560000,4282.0,11.0,3.836667,0.026667,4483,3.541914,27.0
11,12.0,4367,MACIEJ,KARDAS,POL,M,12.0,M30,6.0,1035.0,...,3133.0,16.0,3.566667,4284.0,12.0,3.836667,0.026000,4490,3.547444,32.0
12,13.0,923,SEBASTIAN,CIEŚLA,POL,M,13.0,M20,6.0,1035.0,...,3133.0,14.0,3.566667,4290.0,13.0,3.856667,0.027200,4496,3.552185,28.0
13,14.0,4565,WOJCIECH,DURCZYŃSKI,POL,M,14.0,M30,7.0,1036.0,...,3134.0,17.0,3.566667,4292.0,14.0,3.860000,0.027200,4505,3.559295,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18999,10052.0,3158,PRZEMYSŁAW,STELMACH,POL,M,7129.0,M40,2360.0,2219.0,...,6975.0,10070.0,8.166667,9593.0,10060.0,8.726667,0.089400,10194,8.054041,41.0
19000,10053.0,1987,PAWEŁ,NOWICKI,POL,M,7130.0,M40,2361.0,2220.0,...,6981.0,10076.0,8.186667,9592.0,10057.0,8.703333,0.088267,10194,8.054041,41.0
19003,10056.0,11600,KATARZYNA,KOPROWSKA,POL,K,2925.0,K30,1013.0,2289.0,...,6958.0,10062.0,8.040000,9593.0,10059.0,8.783333,0.079533,10206,8.063522,38.0
19008,10061.0,9801,IWONA,HOŁOWACZ,POL,K,2928.0,K40,990.0,2146.0,...,6877.0,10029.0,8.243333,9594.0,10062.0,9.056667,0.128533,10218,8.073003,49.0


In [15]:
from pycaret.regression import setup, compare_models, finalize_model, plot_model, save_model, predict_model


In [ ]:
# EKSPERYMENT 1: Na całymm zbiorze z zmienną docelową 'Czas'

print("EKSPERYMENT 1: Model (cały zbiór danych)")


exp1 = setup(
    data=model_data,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)


,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 7)"
4,Transformed data shape,"(17626, 7)"
5,Transformed train set shape,"(12338, 7)"
6,Transformed test set shape,"(5288, 7)"
7,Numeric features,5
8,Categorical features,1
9,Preprocess,True


In [ ]:
# Trenuje i ocenia wszystkie modele za pomocą walidacji krzyżowej (domyślnie 10-krotnej).
best_model_v1 = exp1.compare_models(sort='MAE')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,34.8516,2154.8832,46.3755,0.9981,0.0061,0.0047,0.0350
et,Extra Trees Regressor,35.2505,2151.2128,46.3704,0.9981,0.0062,0.0048,0.3180
rf,Random Forest Regressor,35.3739,2152.3368,46.3877,0.9981,0.0062,0.0048,0.5140
en,Elastic Net,35.4956,2016.3966,44.8747,0.9983,0.0060,0.0048,0.0220
lasso,Lasso Regression,35.4989,2016.6405,44.8774,0.9983,0.0060,0.0048,0.2720
lr,Linear Regression,35.5673,2010.0603,44.8055,0.9983,0.0060,0.0049,0.5000
lar,Least Angle Regression,35.5673,2010.0603,44.8055,0.9983,0.0060,0.0049,0.0150
ridge,Ridge Regression,35.5674,2010.0600,44.8055,0.9983,0.0060,0.0049,0.0160
br,Bayesian Ridge,35.5728,2010.3038,44.8083,0.9983,0.0060,0.0049,0.0160
llar,Lasso Least Angle Regression,35.6289,2015.5840,44.8673,0.9983,0.0060,0.0049,0.0160


In [ ]:
# EKSPERYMENT 2: Tylko 5km + wiek + płeć

print("EKSPERYMENT 2: Model (5km + wiek + płeć)")


exp2 = setup(
    data=model_data,
    target='Czas',
    ignore_features=['10 km Czas', '15 km Czas', '20 km Czas'],  # Ignoruj te kolumny
    categorical_features=['Płeć'],
    normalize=True,
    polynomial_features=True,
    polynomial_degree=2,
    remove_outliers=False,
    session_id=123,
    verbose=False,
)


EKSPERYMENT 1: Model PRE-RACE (5km + wiek + płeć)


In [22]:
best_model_v2 = exp2.compare_models(sort='MAE')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,267.8420,133665.3457,365.4182,0.8844,0.0478,0.0357,0.0520
par,Passive Aggressive Regressor,268.8350,137130.2272,370.0971,0.8814,0.0484,0.0357,0.0220
lasso,Lasso Regression,271.5809,130396.8943,360.9701,0.8872,0.0474,0.0364,0.2750
llar,Lasso Least Angle Regression,271.5863,130400.3626,360.9748,0.8872,0.0474,0.0364,0.0180
gbr,Gradient Boosting Regressor,271.7591,130351.4437,360.9148,0.8872,0.0475,0.0364,0.1860
ridge,Ridge Regression,271.9959,131667.1282,362.6873,0.8861,0.0476,0.0365,0.0170
br,Bayesian Ridge,272.0105,131674.8776,362.6977,0.8861,0.0476,0.0365,0.0190
lr,Linear Regression,272.0937,131740.6011,362.7873,0.8861,0.0476,0.0365,0.4810
omp,Orthogonal Matching Pursuit,272.2387,130964.5106,361.7460,0.8867,0.0475,0.0365,0.0200
lar,Least Angle Regression,272.8599,140480.8185,372.9374,0.8785,0.0481,0.0366,0.0220


In [ ]:
# EKSPERYMENT 3: 5km + 10km
print("\n" + "="*60)
print("EKSPERYMENT 3: Model (5km + 10km + wiek + płeć)")
print("="*60)

exp3 = setup(
    data=model_data,
    target='Czas',
    ignore_features=['20 km Czas'],  # Ignoruj tylko 20km
    categorical_features=['Płeć'],
    normalize=True,
    polynomial_features=True,
    polynomial_degree=2,
    remove_outliers=False,
    session_id=124,
    verbose=False,
)



EKSPERYMENT 2: Model (5km + 10km + wiek + płeć)


In [27]:
best_model_v3 = exp3.compare_models(sort='MAE')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,79.5760,13232.2147,114.9540,0.9884,0.0150,0.0107,0.0220
lr,Linear Regression,79.5940,13227.6861,114.9375,0.9884,0.0150,0.0107,0.0260
huber,Huber Regressor,80.4840,14139.8538,118.8077,0.9876,0.0154,0.0107,0.1250
ridge,Ridge Regression,80.8076,13790.0796,117.3328,0.9879,0.0153,0.0108,0.0220
llar,Lasso Least Angle Regression,82.9320,14196.6957,119.0437,0.9876,0.0156,0.0111,0.0240
rf,Random Forest Regressor,83.6759,14281.5691,119.4177,0.9875,0.0157,0.0112,1.5760
lasso,Lasso Regression,83.9359,14400.4016,119.8945,0.9874,0.0157,0.0113,0.0450
lightgbm,Light Gradient Boosting Machine,84.4088,14323.7652,119.5931,0.9874,0.0157,0.0114,0.1520
et,Extra Trees Regressor,84.5200,14703.1458,121.1804,0.9871,0.0159,0.0114,0.6930
par,Passive Aggressive Regressor,85.7395,15530.2163,124.4335,0.9864,0.0162,0.0115,0.0280


In [ ]:
# EKSPERYMENT 4: 5km + 10km + 15km 
print("EKSPERYMENT 4: (5km + 10km + 15km + wiek + płeć)")

exp4 = setup(
    data=model_data,
    target='Czas',
    ignore_features=['20 km Czas'],  # Ignoruj tylko 20km
    categorical_features=['Płeć'],
    normalize=True,
    polynomial_features=True,
    polynomial_degree=2,
    remove_outliers=False,
    session_id=124,
    verbose=False,
)


EKSPERYMENT 2: (5km + 10km + 15km + wiek + płeć)


In [30]:
best_model_v4 = exp4.compare_models(sort='MAE')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,79.5760,13232.2147,114.9540,0.9884,0.0150,0.0107,0.0290
lr,Linear Regression,79.5940,13227.6861,114.9375,0.9884,0.0150,0.0107,0.0220
huber,Huber Regressor,80.4840,14139.8538,118.8077,0.9876,0.0154,0.0107,0.1250
ridge,Ridge Regression,80.8076,13790.0796,117.3328,0.9879,0.0153,0.0108,0.0240
llar,Lasso Least Angle Regression,82.9320,14196.6957,119.0437,0.9876,0.0156,0.0111,0.0210
rf,Random Forest Regressor,83.6759,14281.5691,119.4177,0.9875,0.0157,0.0112,1.6290
lasso,Lasso Regression,83.9359,14400.4016,119.8945,0.9874,0.0157,0.0113,0.0570
lightgbm,Light Gradient Boosting Machine,84.4088,14323.7652,119.5931,0.9874,0.0157,0.0114,0.1580
et,Extra Trees Regressor,84.5200,14703.1458,121.1804,0.9871,0.0159,0.0114,0.6960
par,Passive Aggressive Regressor,85.7395,15530.2163,124.4335,0.9864,0.0162,0.0115,0.0330


In [31]:
# EKSPERYMENT 5: WSZYSTKIE dystanse (5 + 10 + 15 + 20km)

print("EKSPERYMENT 5: Model (wszystkie dystanse)")
exp5 = setup(
    data=model_data,
    target='Czas',
    # Brak ignore_features - używamy WSZYSTKIEGO!
    categorical_features=['Płeć'],
    normalize=True,
    polynomial_features=True,
    polynomial_degree=2,
    remove_outliers=False,
    session_id=125,
    verbose=False,
)


EKSPERYMENT 5: Model (wszystkie dystanse)


In [32]:
best_model_v5 = exp5.compare_models(sort='MAE')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,33.9395,1971.9242,44.3337,0.9983,0.0059,0.0046,0.0260
br,Bayesian Ridge,34.0040,1976.4536,44.3841,0.9983,0.0059,0.0046,0.0260
huber,Huber Regressor,34.5267,2110.8385,45.9110,0.9982,0.0061,0.0047,0.1740
ridge,Ridge Regression,34.9889,1987.6848,44.5528,0.9983,0.0059,0.0048,0.0230
et,Extra Trees Regressor,35.4338,2185.6015,46.7377,0.9981,0.0062,0.0048,0.8640
rf,Random Forest Regressor,35.7165,2156.9375,46.4272,0.9981,0.0062,0.0049,2.1520
lightgbm,Light Gradient Boosting Machine,36.6681,2133.2666,46.1704,0.9982,0.0062,0.0050,0.2050
llar,Lasso Least Angle Regression,38.2776,2238.4898,47.2926,0.9981,0.0063,0.0052,0.0250
lasso,Lasso Regression,38.2794,2233.7356,47.2426,0.9981,0.0063,0.0052,0.0560
omp,Orthogonal Matching Pursuit,38.2863,2248.1410,47.3954,0.9981,0.0063,0.0052,0.0240


In [33]:
# EKSPERYMENT 6: Tylko wiek i płeć (bez czasu 5km!)

print("EKSPERYMENT 4: Model (tylko wiek + płeć)")
exp6 = setup(
    data=model_data,
    target='Czas',
    ignore_features=['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas'],
    categorical_features=['Płeć'],
    normalize=True,
    polynomial_features=True,
    polynomial_degree=2,
    remove_outliers=False,
    session_id=126,
    verbose=False,
)



EKSPERYMENT 4: Model (tylko wiek + płeć)


In [34]:
best_model_v6 = exp6.compare_models(sort='MAE')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,807.1892,1016442.8952,1007.9883,0.1204,0.1402,0.1139,0.0610
lightgbm,Light Gradient Boosting Machine,809.3038,1021343.7291,1010.4240,0.1161,0.1404,0.1141,0.1050
et,Extra Trees Regressor,809.5016,1021902.0947,1010.6971,0.1156,0.1405,0.1142,0.0570
dt,Decision Tree Regressor,809.5865,1022092.1224,1010.7901,0.1155,0.1405,0.1142,0.0170
rf,Random Forest Regressor,809.6511,1022220.3686,1010.8541,0.1154,0.1405,0.1142,0.0860
huber,Huber Regressor,811.0674,1031592.0427,1015.4414,0.1074,0.1407,0.1137,0.0210
par,Passive Aggressive Regressor,811.5751,1035368.2035,1017.3012,0.1041,0.1408,0.1135,0.0170
lasso,Lasso Regression,812.4057,1028988.9826,1014.1555,0.1097,0.1410,0.1146,0.0190
llar,Lasso Least Angle Regression,812.4059,1028993.5687,1014.1577,0.1097,0.1410,0.1146,0.0150
br,Bayesian Ridge,812.4710,1029031.6947,1014.1794,0.1096,0.1410,0.1146,0.0160


In [36]:
best_model = compare_models([best_model_v1, best_model_v2, best_model_v3, best_model_v4, best_model_v5])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
2,Bayesian Ridge,812.4710,1029031.6947,1014.1794,0.1096,0.1410,0.1146,0.0180
3,Bayesian Ridge,812.4710,1029031.6947,1014.1794,0.1096,0.1410,0.1146,0.0160
4,Linear Regression,812.4921,1029049.6821,1014.1898,0.1096,0.1410,0.1146,0.0160
0,Huber Regressor,811.0674,1031592.0427,1015.4414,0.1074,0.1407,0.1137,0.0250
1,Huber Regressor,811.0674,1031592.0427,1015.4414,0.1074,0.1407,0.1137,0.0230


Wybieram najlepszy model z najmniejszym błędem

In [37]:
# Strojenie hiperparametrów
best_model_tuned = exp.tune_model(
    estimator=best_model,  # Wybrany model
    n_iter=50,            # Liczba iteracji dla Random Grid Search
    optimize='MAE'         # Metryka do optymalizacji
)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,35.3063,1968.9716,44.3731,0.9983,0.0059,0.0048
1,34.1253,1879.1742,43.3494,0.9984,0.0058,0.0047
2,34.8646,1844.6810,42.9497,0.9983,0.0058,0.0048
3,34.6871,1877.6693,43.3321,0.9983,0.0058,0.0048
4,36.2360,2382.3923,48.8098,0.9980,0.0064,0.0049
5,35.4766,2018.7952,44.9310,0.9982,0.0060,0.0048
6,35.6965,1996.5646,44.6829,0.9983,0.0059,0.0048
7,36.7886,2032.1082,45.0789,0.9983,0.0060,0.0050
8,35.9888,1997.1608,44.6896,0.9984,0.0060,0.0049


Fitting 10 folds for each of 50 candidates, totalling 500 fits


In [38]:
best_final_model = compare_models([best_model, best_model_tuned])


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Bayesian Ridge,812.4710,1029031.6947,1014.1794,0.1096,0.1410,0.1146,0.0160
1,Bayesian Ridge,812.4710,1029031.6947,1014.1794,0.1096,0.1410,0.1146,0.0150


## Porównanie oryginalnego modelu, oraz modelu po strojeniu hiperparametrów

In [39]:
exp.predict_model(best_final_model)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 10 km Czas
- 15 km Czas
- 20 km Czas
- 5 km Czas
Feature names seen at fit time, yet now missing:
- Płeć^2
- Wiek Płeć
- Wiek^2


Podgląd predykcji pozwala zrozumieć, co model przewidywał i na podstawie jakich danych:

Cel Predykcji: Zmienna docelowa to prediction_label. Biorąc pod uwagę inne kolumny (5 km Czas Open, 10 km Czas Open, etc.), można ztwierdzić że model przewiduje czas biegu (np. półmaratonu/maratonu).

Kluczowe Predyktory: Model wykorzystuje szereg cech związanych z bieganiem, takich jak:

Czasy na krótszych dystansach (5 km Czas Open, 10 km Czas Open, 15 km Czas Open).

Płeć, Kategoria wiekowa (M20, K20, M30).

In [ ]:
plot_model(best_final_model, plot='feature')
plot_model(best_final_model, plot='error')
plot_model(best_final_model, plot='residuals')


In [ ]:
# Trenowanie modelu na całym zbiorze danych
final_model = finalize_model(best_final_model)


In [ ]:
# Zapis modelu lokalnie
final_model = finalize_model(best_model)
save_model(final_model, 'final_regression_pipeline')


In [ ]:
s3.upload_file(
    Filename='final_regression_pipeline.pkl',
    Bucket=BUCKET_NAME,
    Key='Train_Model/final_regression_pipeline.pkl'
)


In [ ]:
def predict_halfmarathon_time(wiek, plec, five_k_time_sec, model):
    """
    Funkcja do predykcji czasu ukończenia półmaratonu.

    Args:
        wiek (int): wiek biegacza
        plec (str): 'M' lub 'K'
        five_k_time_sec (int): czas 5 km w sekundach
        model: wytrenowany model pycaret

    Returns:
        str: czas w formacie H:M:S
    """
    # Dane wejściowe z wszystkimi kolumnami
    user_input = pd.DataFrame([{
        'Miejsce': 1.0,
        'Numer startowy': 1,
        'Imię': 'Test',
        'Nazwisko': 'User',
        'Kraj': 'POL',
        'Płeć': plec,
        'Płeć Miejsce': 1.0,
        'Kategoria wiekowa': 'M30' if plec == 'M' else 'K30',
        'Kategoria wiekowa Miejsce': 1.0,
        '5 km Czas': five_k_time_sec,
        '5 km Miejsce Open': 1.0,
        '10 km Czas': five_k_time_sec * 2,  # Przybliżenie
        '10 km Miejsce Open': 1.0,
        '15 km Czas': five_k_time_sec * 3,  # Przybliżenie
        '15 km Miejsce Open': 1.0,
        '20 km Czas': five_k_time_sec * 4,  # Przybliżenie
        '20 km Miejsce Open': 1.0,
        'Tempo Stabilność': 0.02,
        'Wiek': wiek
    }])

    # Tworzenie cech interakcyjnych (muszą być tworzone tak samo jak w treningu)
    # Normalizacja czasu 5km
    user_input['czas_5km_normalized'] = (user_input['5 km Czas'] - mean_5km) / std_5km
    
    # Normalizacja wieku
    user_input['wiek_normalized'] = (user_input['Wiek'] - mean_wiek) / std_wiek
    
    # Zmienna binarna dla płci
    user_input['is_male'] = (user_input['Płeć'] == 'M').astype(int)
    
    # Cechy interakcyjne
    user_input['czas5km_x_wiek'] = user_input['czas_5km_normalized'] * user_input['wiek_normalized']
    user_input['czas5km_x_male'] = user_input['czas_5km_normalized'] * user_input['is_male']
    user_input['czas5km_x_female'] = user_input['czas_5km_normalized'] * (1 - user_input['is_male'])
    user_input['wiek_x_male'] = user_input['wiek_normalized'] * user_input['is_male']
    user_input['czas5km_x_wiek_x_male'] = user_input['czas_5km_normalized'] * user_input['wiek_normalized'] * user_input['is_male']
    
    # Cechy nieliniowe
    user_input['czas_5km_squared'] = user_input['czas_5km_normalized'] ** 2
    user_input['wiek_squared'] = user_input['wiek_normalized'] ** 2
    
    # Interakcje z cechami kwadratowymi
    user_input['czas5km_sq_x_wiek'] = user_input['czas_5km_squared'] * user_input['wiek_normalized']
    user_input['czas5km_x_wiek_sq'] = user_input['czas_5km_normalized'] * user_input['wiek_squared']

    # Predykcja
    prediction = predict_model(model, data=user_input)
    pred_seconds = int(round(prediction['prediction_label'][0]))

    # Konwersja na H:M:S
    pred_time_str = str(datetime.timedelta(seconds=pred_seconds))
    return pred_time_str

# Przykład użycia:
wiek_input = 42
plec_input = 'M'
czas_5km_input = 2000  # w sekundach

predykcja_czasu = predict_halfmarathon_time(wiek_input, plec_input, czas_5km_input, final_model)
print(f"Przewidywany czas ukończenia półmaratonu: {predykcja_czasu}")



In [ ]:
# Zapisz statystyki normalizacji
normalization_stats = {
    'mean_5km': float(mean_5km),
    'std_5km': float(std_5km),
    'mean_wiek': float(mean_wiek),
    'std_wiek': float(std_wiek)
}

import json

# Zapisz lokalnie
with open('normalization_stats.json', 'w') as f:
    json.dump(normalization_stats, f, indent=4)

print("✓ Statystyki normalizacji zapisane lokalnie")

# Prześlij do S3
try:
    s3.upload_file(
        Filename='normalization_stats.json',
        Bucket=BUCKET_NAME,
        Key='Train_Model/normalization_stats.json'
    )
    print("✓ Statystyki normalizacji przesłane do S3")
except Exception as e:
    print(f"❌ Błąd przesyłania statystyk do S3: {e}")

